# **Objetivo del notebook**

En esta etapa vamos a limpiar y preparar los datos: realizaremos una limpieza exhaustiva de los comentarios de texto, eliminando caracteres especiales, convirtiendo el texto a minúsculas y tokenizando las palabras.

Como estamos trabajando con textos y vamos entrenar un modelo de NLP, el preprocesamiento de los datos en este caso consistirá en distintos pasos de limpieza del texto para dejarlo listo para el modelo.



# **Contenido del Notebook**

1. Ingeniería de características
2. Eliminar columnas innecesarias
3. Regex: Limpieza y filtrado de texto
4. Stopwords: Eliminación de palabras vacías
5. Tokenización: Divide los comentarios en palabras o tokens.
6. Lematización: Normalización de las palabras
7. Vectorizar: ^_^ 
8. Codificación: Convertir Bool to int

In [1]:
# Data analysis libraries
import numpy as np
import pandas as pd
import math
from scipy import stats
from tabulate import tabulate

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Data cleaning
import re
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-06 13:28:32.573578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 13:28:32.700350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-06 13:28:32.700384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-06 

In [2]:
# read csv file
path = "/work/youtoxic_english_1000.csv"
df = pd.read_csv(path)

## **1. Ingeniería de características**

Inicialmente vamos a usar 'IsToxic' como variable objetivo.
En una siguiente vuelta de Scrum queremos probar a crear una nueva columna a partir de los 'True' de las ya existentes, y usarla como variable objetivo.

## **2. Eliminar columnas innecesarias**

## **3. Regex: Limpieza y filtrado de texto**

In [3]:
def limpieza_regex(text):
  text = str(text).lower() # Convierte el texto a minúsculas
  #text = re.sub('\n', '', text)  # Elimina saltos de línea
  text = re.sub('\w*\d\w*', '', text)  # Elimina palabras que contienen números
  #text = re.sub('\[.*?\]', '', text) # Elimina texto entre corchetes
  text = re.sub('\[|\]', '', text) # Elimina corchetes
  text = re.sub(r"\@w+|\#w+",'',text)  # Elimina menciones y hashtags de redes sociales
  text = re.sub('https?://\S+|www\.\S+', '', text)  # Elimina URLs que comiencen con http, https o www
  text = re.sub('<.*?>+', '', text)  # Elimina etiquetas HTML
  text = re.sub(r"[^\w\s]",'',text)  # Elimina caracteres no alfanuméricos
  #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  # Elimina signos de puntuación
  text = text.strip() # Elimina espacios antes y despues


  # Elimina emojis
  #emoji_pattern = emoji.get_emoji_regexp()
  #text = emoji_pattern.sub('', text)
    
  return text

In [4]:
df['Text'] = df['Text'].apply(limpieza_regex)

In [5]:
# Reemplazamos los xa0 (que en unicode repreentan espacios) por espacios normales
df['Text'] = df['Text'].replace('\xa0', ' ')

In [6]:
df.to_csv("Youtube_limpio_version4.csv", index=False)

In [7]:

 " michael smerconish hack " 
"     leaving out information when giving a police report because they didnt ask you for that info is lying by omission thats why its not outright lyingbecause you omitted the information these are some stupid lawyers"
"moly loooooooooooooooooooooooves cops  there are other witnesses  also clueless about the riots  also about the decades of police violence and repression in the areaalso ignorant of the effects of marijuana"
"right on finally some sane opinion on this event these black bastards are soooooooooo ignorant running around yelling and looting and carrying on just like in la riots same old shit"
"blacklivesdontmatter"
"a thug going to college  black people  are always  crying    what about the american indians  i dont see no one crying for them  "
"xdddddddddddddddd"
"lmfao"

'lmfao'

## **4. Stopwords: Eliminación de palabras vacías**

In [8]:
!python -m spacy download en_core_web_sm > /dev/null 2>&1

In [9]:
# Instanciamos el modelo
nlp_english = spacy.load("en_core_web_sm")

In [10]:
# Funcion para eliminar Stopwords
def eliminar_stop_words(text):
    doc = nlp_english(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(token for token in tokens)

In [11]:
# Aplicamos la funcion en la columna objetivo 
df['Text'] = df['Text'].apply(eliminar_stop_words)

In [12]:
df.head(10)

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,people step case nt people situation lump m...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,law enforcement trained shoot apprehend trai...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,nt reckon black lives matter banners held whit...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,large number people like police officers calle...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,arab dude absolutely right shot extra time s...,False,False,False,False,False,False,False,False,False,False,False,False
5,Ugg9a6FtoXdxmXgCoAEC,04kJtp6pVXI,people facebook ties isis terrorist groups ...,True,False,False,False,False,True,False,False,False,False,True,False
6,Ugga9KzkNDGvlXgCoAEC,04kJtp6pVXI,check tube post black man goes epic rant fergu...,True,False,False,False,False,True,True,False,False,False,False,False
7,UggBlIXoph7p-3gCoAEC,04kJtp6pVXI,love pussy staten island spit cop love happen ...,True,True,False,False,True,True,True,False,False,False,False,False
8,UggD1aYSn7KOR3gCoAEC,04kJtp6pVXI,agree protestor,False,False,False,False,False,False,False,False,False,False,False,False
9,UggGm8a1fu8brngCoAEC,04kJtp6pVXI,mike browns father boooshit,True,True,False,False,True,False,False,False,False,False,False,False


In [13]:
df.to_csv("Youtube_limpio_stop_words.csv", index=False)

## **5. Tokenización: Divide los comentarios en palabras o tokens**

In [18]:
def tokenizar_texto(text):
    doc = nlp_english(text)
    return [token.text for token in doc]

In [19]:
# Aplicar la función a la columna 'Text'
df['Tokens'] = df['Text'].apply(tokenizar_texto)

In [22]:
# Función para filtrar los tokens que contienen \xa0 o espacios en blanco
def filtrar_tokens(tokens):
    return [token for token in tokens if token.strip() and '\xa0' not in token]

# Aplicar la función a la columna 'Tokens'
df['Tokens'] = df['Tokens'].apply(filtrar_tokens)

In [23]:
# Imprimir tokens para el primer registro, por ejemplo
print(df['Tokens'][0])

['people', 'step', 'case', 'nt', 'people', 'situation', 'lump', 'mess', 'matters', 'hands', 'makes', 'kinds', 'protests', 'selfish', 'rational', 'thought', 'investigation', 'guy', 'video', 'heavily', 'emotional', 'hyped', 'wants', 'heard', 'gets', 'heard', 'presses', 'reasonable', 'discussion', 'kudos', 'smerconish', 'keeping', 'level', 'time', 'letting', 'masri', 'fool', 'dare', 'tore', 'city', 'protest', 'dishonor', 'entire', 'incident', 'hate', 'way', 'police', 'brutality', 'epidemic', 'wish', 'stop', 'pretending', 'like', 'knew', 'exactly', 'going', 's', 'measurable', 'people', 'honestly', 'witnessed', 'incident', 'clue', 'way', 'issue', 'swung', 'grand', 'jury', 'informed', 'trust', 'majority', 'rule', 'right', 'course', 'action', 'let', 'thank', 'police', 'officers', 'america', 'actually', 'serve', 'protect', 'bit', 'jerk', 'pull', 'respect', 'job', 'know', 'people', 'going', 'pout', 'held', 'accountable', 'actions', 'people', 'hate', 'police', 'need', 'officer', 'emergency']


## **6. Lematización: Normalización de las palabras**

## **7. Vectorizar**

## **8. Codificación: Convertir Bool to int**

##

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=22be43cf-6e58-4d4e-8694-3183dc7644ab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>